# Introduction: A Working MNIST Classifier

- Understanding softmax and NLL: https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/
- Visualizing MNIST (Chris Olah): http://colah.github.io/posts/2014-10-Visualizing-MNIST/
- Pytorch: https://pytorch.org/
- Pytorch samples: https://github.com/pytorch/examples

In [1]:
import torch as t
import torch.nn as nn
import torch.nn.functional as f
import torchvision as tv

![](img/mnist.png)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        # [64, 1, 28, 28]
        x = f.relu(self.conv1(x)) # [64, 20, 24, 24]
        x = f.max_pool2d(x, 2, 2) # [64, 20, 12, 12]
        x = f.relu(self.conv2(x)) # [64, 50, 8, 8]
        x = f.max_pool2d(x, 2, 2) # [64, 50, 4, 4]
        x = x.view(-1, 4*4*50) # [64, 800]
        x = f.relu(self.fc1(x)) # [64, 500]
        x = self.fc2(x) # [64, 10]
        x = f.log_softmax(x, dim=1) # [64, 10]
        return x

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = f.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # item(): https://pytorch.org/docs/stable/tensors.html#torch.Tensor.item
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with t.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += f.nll_loss(output, target, reduction='sum').item() # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # Get the index of the max log-probability
            # eq(): https://pytorch.org/docs/stable/torch.html#torch.eq
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

transform = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.1307,), (0.3081,))])

train_set = tv.datasets.MNIST('.', train=True, transform = transform)
test_set = tv.datasets.MNIST('.', train=False, transform = transform)
train_loader = t.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = t.utils.data.DataLoader(test_set, batch_size=1000, shuffle=True)

model = Net().to(device)
optimizer = t.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1, 10 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.294235
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.287354
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.239572
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.233439
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.199394
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.140323
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.093626
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.995074
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.918356
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.712761
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.558403
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.404655
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.021049
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.943486
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.973831
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.632423
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.757206
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.634556
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.746596
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.533116
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.283630
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.120316
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.163428
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.256094
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.096488
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.024944
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.091017
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.061440
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.176742
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.175559
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.043549
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.129496
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.130572
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.170471
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.086109
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.121983
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.069000
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.072503
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.103068
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.060810


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.091667
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.038338
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.066385
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.045657
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.033979
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.025371
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.053628
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.152319
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.088246
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.052708
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.246636
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.123200
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.020107
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.108814
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.023436
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.026357
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.025603
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.062732
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.103704
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.094462


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.091121
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.038588
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.027128
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.049715
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.194384
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.071428
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.012822
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.164458
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.016081
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.047865
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.175228
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.045216
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.023902
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.138973
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.016976
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.027105
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.041718
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.019982
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.024428
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.032147


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.027464
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.027158
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.087735
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.022167

Test set: Average loss: 0.0413, Accuracy: 9869/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.014806
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.013444
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.034111
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.006761
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.006902
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.031775
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.135541
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.018855
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.081306
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.019996
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.065823
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.016766
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.028647
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.004617
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.022222
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.005489
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.012012
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.016253
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.015174
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.056932
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.044335
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.051487
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.016451
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.026465
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.030435
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.006606
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.097225
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.098927
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.017409
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.083905
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.020682
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.069495
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.006126
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.062793
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.061069
